In [1]:
import cclib
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants

from overreact import core, _thermo, rates, simulate

%matplotlib inline

In [2]:
jmol = constants.physical_constants["Hartree energy"][0] * constants.N_A
jmol

2625499.6394798253

I want to reproduce the results of [this paper](https://doi.org/10.1002/qua.25686):

$$\require{mhchem}\ce{CH4 + Cl^{.} <=> RC -> RC^{\ddagger} <=> CH3^{.} + HCl}$$

In [3]:
scheme = core.parse_reactions("""
    CH4 + Cl. <=> RC -> RC* -> PC <=> CH3. + HCl
""")
scheme

Scheme(compounds=('CH4', 'Cl.', 'RC', 'RC*', 'PC', 'CH3.', 'HCl'), reactions=('CH4 + Cl. -> RC', 'RC -> CH4 + Cl.', 'RC -> RC*', 'RC* -> PC', 'PC -> CH3. + HCl', 'CH3. + HCl -> PC'), is_half_equilibrium=(True, True, False, False, True, True), A=((-1.0, 1.0, 0.0, 0.0, 0.0, 0.0), (-1.0, 1.0, 0.0, 0.0, 0.0, 0.0), (1.0, -1.0, -1.0, 0.0, 0.0, 0.0), (0.0, 0.0, 1.0, -1.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0, -1.0, 1.0), (0.0, 0.0, 0.0, 0.0, 1.0, -1.0), (0.0, 0.0, 0.0, 0.0, 1.0, -1.0)), B=((-1.0, 0.0, 0.0, 0.0, 0.0, 0.0), (-1.0, 0.0, 0.0, 0.0, 0.0, 0.0), (1.0, 0.0, -1.0, 0.0, 0.0, 0.0), (0.0, 0.0, 1.0, -1.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0, -1.0, 0.0), (0.0, 0.0, 0.0, 0.0, 1.0, 0.0), (0.0, 0.0, 0.0, 0.0, 1.0, 0.0)))

Now let's read some data from files:

In [4]:
# TODO(schneiderfelipe): correct file paths
logfiles = {
    "CH4": cclib.ccopen("../../data/tanaka1996/UMP2/6-311G\(2df,2pd\)/methane.out").parse().getattributes(),
    "RC": cclib.ccopen("../../data/Dzib_2018/reactants.out").parse().getattributes(),
    "RC*": cclib.ccopen("../../data/Dzib_2018/ts.out").parse().getattributes(),
    "PC": cclib.ccopen("../../data/Dzib_2018/products.out").parse().getattributes(),
    "CH3.": cclib.ccopen("../../data/Dzib_2018/ch3.out").parse().getattributes(),
    "HCl": cclib.ccopen("../../data/Dzib_2018/hcl.out").parse().getattributes(),
#     "Cl.": cclib.ccopen("../../data/Dzib_2018/cl.out").parse().getattributes(),
    "Cl.": {
        "enthalpy": -459.62856787 - 0.00283260,
        "entropy": 0.00065446 + 0.01740262,  # S(el) + S(vib) + S(rot) + S(trans)
    },
}
logfiles["Cl."]["freeenergy"] = logfiles["Cl."]["enthalpy"] - logfiles["Cl."]["entropy"]

freeenergy = np.array([logfiles[compound]["freeenergy"] for compound in scheme.compounds])
freeenergy

AttributeError: 'NoneType' object has no attribute 'parse'

In [ ]:
Gconc = constants.R * 298.15 * np.log(constants.atm / (constants.R * 298.15))
Gconc / 4184.0

The below is just to remind myself that the enthalpy and entropy come out subtractively contributing to the free energy in ORCA!

In [ ]:
logfiles["CH4"]["enthalpy"] - logfiles["CH4"]["entropy"] == logfiles["CH4"]["freeenergy"]

Some barriers, which will then be adjusted for molecularity (if needed, it's easier to adjust the absolute Gibbs energies by ):

In [ ]:
barriers = thermo.get_delta(scheme.B, freeenergy * jmol + Gconc)
barriers / 4184

In [ ]:
molecularity = np.sum(np.where(scheme.A < 0, -scheme.A, 0), axis=0)
molecularity

In [ ]:
k = rates.eyring(barriers)
k

In [ ]:
1e-6 * k[2] * k[0] / k[1]

In [ ]:
scheme.reactions

In [ ]:
dydt = simulate.get_dydt(scheme, k)
t, y = simulate.get_y(dydt, y0=[1.0, 1.0, 0.0, 0.0, 0., 0., 0.], t_span=[0.0, 1.0e-4])

for i, compound in enumerate(scheme.compounds):
    if not compound.endswith("*"):
        plt.plot(t, y[i], label=compound)
plt.legend()

y[:, -1]

In [ ]:
dydt_vals = np.array([dydt(t, yt) for yt in y.T])

for i, compound in enumerate(scheme.compounds):
    if not compound.endswith("*"):
        plt.plot(t, dydt_vals.T[i], label=compound)
plt.legend()

plt.ylim(-5e3, 5e3)